<a href="https://colab.research.google.com/github/Cholico/Proyectos-de-DataCamp-/blob/main/Depuraci%C3%B3n_de_un_flujo_de_trabajo_de_datos_de_ventas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Como ingeniero de datos, a menudo se enfrenta a desafíos inesperados en los flujos de trabajo. En este escenario, la función `load_and_check()`, a cargo de administrar los datos de ventas, encuentra problemas después de la última actualización. Desafortunadamente, su colega que generalmente maneja este código está actualmente de vacaciones, lo que lo deja a usted solo para solucionar los problemas.

Su tarea es identificar y abordar los problemas en el flujo de datos de ventas **sin analizar cada línea de código**. La función `load_and_check()` carga el conjunto de datos `sales.csv` y realiza varias comprobaciones. Inicialmente, verifica la forma del conjunto de datos, asegurándose de que coincida con las expectativas. Posteriormente, se realizan comprobaciones de integridad para mantener la coherencia de los datos y marcar cualquier anomalía.

El conjunto de datos `sales.csv` tiene varias columnas, que se centran en campos críticos como `Total`, `Cantidad`, `Precio unitario`, `Impuesto` y `Fecha`. Es fundamental que la columna "Impuestos" represente con precisión el 5 % del subtotal, calculado a partir del "Precio unitario" multiplicado por la "Cantidad".

**Su objetivo es resolver los problemas de la canalización, con el objetivo de que el código devuelva 2 mensajes de éxito al finalizar.** Mientras tanto, intente mantener la estructura original tanto como sea posible. Cambie las columnas existentes solo si es necesario y asegúrese de que los datos sigan siendo precisos. Tenga en cuenta la actualización de las declaraciones if relevantes en los controles según sea necesario.

In [3]:
import pandas as pd

In [29]:

def load_and_check():
    # Step 1: Load the data and check if it has the expected shape
    data = pd.read_csv('sales.csv')

    if data.shape[1] != 17:
        print("Please check that the data was loaded properly!")
    else:
        print("Data loaded successfully.")

    # Step 2: Calculate statistical values and merge with the original data
    grouped_data = data.groupby(['Date'])['Total'].agg(['mean', 'std'])
    grouped_data['threshold'] = 3 * grouped_data['std']
    grouped_data['max'] = grouped_data['mean'] + grouped_data.threshold
    grouped_data['min'] = grouped_data[['mean', 'threshold']].apply(lambda row: max(0, row['mean'] - row['threshold']), axis=1)
    data = pd.merge(data, grouped_data, on='Date', how='left')

    # Condition_1 checks if 'Total' is within the acceptable range (min to max) for each date
    data['Condition_1'] = (data['Total'] >= data['min']) & (data['Total'] <= data['max'])
    data['Condition_1'].fillna(False, inplace=True)

    # Condition_2 checks if the 'Tax' column is properly calculated as 5% of (Quantity * Unit price)
    data['Condition_2'] = round(data['Quantity'] * data['Unit price'] * 0.05, 1) == round(data['Tax'], 1)


    # Step 3: Check if all rows pass both Condition_1 and Condition_2
    # Success indicates data integrity; failure suggests potential issues.
    if (data['Condition_1'].sum() == data.shape[0]) and (data['Condition_2'].sum() == data.shape[0]):
        print("Data integrity check was successful! All rows pass the integrity conditions.")
    else:
        print("Something fishy is going on with the data! Integrity check failed for some rows!")

    return data

processed_data = load_and_check()

Please check that the data was loaded properly!
Something fishy is going on with the data! Integrity check failed for some rows!


In [ ]:
import pandas as pd

def load_and_check():
    # Step 1: Load the data and check if it has the expected shape
    data = pd.read_csv('sales.csv')

    # Issue 1 fixed: Correct number of expected columns
    if data.shape[1] != 17:
        print("Please check that the data was loaded properly!")
    else:
        print("Data loaded successfully.")

    # Step 2: Calculate statistical values and merge with the original data
    grouped_data = data.groupby(['Date'])['Total'].agg(['mean', 'std'])
    grouped_data['threshold'] = 3 * grouped_data['std']
    grouped_data['max'] = grouped_data['mean'] + grouped_data.threshold
    grouped_data['min'] = grouped_data[['mean', 'threshold']].apply(lambda row: max(0, row['mean'] - row['threshold']), axis=1)
    data = pd.merge(data, grouped_data, on='Date', how='left')

    # Issue 2 fixed:  Recalculating the 'Tax' column
    data['Tax'] = (data['Quantity'] * data['Unit price']).astype(float) * 0.05  # Assuming tax is 5% of the subtotal

    # Condition_1 checks if 'Total' is within the acceptable range (min to max) for each date
    data['Condition_1'] = (data['Total'] >= data['min']) & (data['Total'] <= data['max'])
    data['Condition_1'].fillna(False, inplace=True)

    # Condition_2 checks if the 'Tax' column is properly calculated as 5% of (Quantity * Unit price)
    data['Condition_2'] = round(data['Quantity'] * data['Unit price'] * 0.05, 1) == round(data['Tax'], 1)

    # Step 3: Check if all rows pass both Condition_1 and Condition_2
    # Success indicates data integrity; failure suggests potential issues.
    if (data['Condition_1'].sum() == data.shape[0]) and (data['Condition_2'].sum() == data.shape[0]):
        print("Data integrity check was successful! All rows pass the integrity conditions.")
    else:
        print("Something fishy is going on with the data! Integrity check failed for some rows!")

    return data

processed_data = load_and_check()